In [1]:
# -*- coding: utf-8 -*-
import sys
import os       
import pandas as pd
import datetime

os.chdir('D:/zxdf/Workspaces/PycharmProjects/flask_ml')
from app import config, Elasticsearch_Util
#from app.lib import nltk_text_analyze_lib as textAnalyzer
#from app.lib import sklearn_classification_lib as skclassification

starttime = datetime.datetime.now()
print ("当前的日期和时间是 %s" % starttime)

In [2]:
def getMoreDataFromES(self, index, doc_type,beginTime, endTime, source_include,size):
    querybody = {
        "query": {
            "match_all": {}
        },
        "_source": {
            "includes": source_include,
            "excludes": []
        }
    }
    # Initialize the scroll
    print "Initialize the scroll"
    page = self.es.search(
        index=index,
        doc_type=doc_type,
        scroll='2m',
        size=size,
        body=querybody)
    sid = page['_scroll_id']
    scroll_size = page['hits']['total']
    
    print "sid: " + str(sid)
    print "scroll size: " + str(scroll_size)
    
    records = []
    for doc in page['hits']['hits']:
            source=doc['_source']
            source["id"]=doc['_id']
            records.append(source)
    
    df = None
   #Start scrolling
    while (scroll_size > 0):
        print "Scrolling..."
        page = self.es.scroll(scroll_id=sid, scroll='2m')
        # Update the scroll ID
        sid = page['_scroll_id']
        print "sid: " + str(sid)
        # Get the number of results that we returned in the last scroll
        scroll_size = len(page['hits']['hits'])
        
        for doc in page['hits']['hits']:
            source=doc['_source']
            source["id"]=doc['_id']
            records.append(source)
            
        print "scroll size: " + str(scroll_size)
        
    if records:
        df = pd.DataFrame(records) 
        # Do something with the obtained page
    return df

In [3]:
index = "1012-knowledge_training_normal-20180131"
es_type = "knowledge_training_normal"
beginTime = "2018-01-26T00:00:00.000+08:00"
endTime = "2018-01-27T00:00:00.000+08:00"
size = 10000
tag = "isException"
logmsg = "logBody"
source_include = [tag, logmsg]

es_util = Elasticsearch_Util()
train_data_normal = getMoreDataFromES(es_util, index=index, doc_type=es_type,
                           beginTime=beginTime, endTime=endTime,
                           size=size,
                           source_include=source_include)

In [4]:
len(train_data_normal)

1723

In [5]:
index = "1012-knowledge_training_error-20180131"
es_type = "knowledge_training_error"
beginTime = "2018-01-30T00:00:00.000+08:00"
endTime = "2018-02-01T00:00:00.000+08:00"
fromnum = 0
size = 10000
#1085  804:281
tag = "isException"
logmsg = "logBody"
source_include = [tag, logmsg]

es_util = Elasticsearch_Util()
train_data_error = getMoreDataFromES(es_util, index=index, doc_type=es_type,
                           beginTime=beginTime, endTime=endTime,
                           size=size,
                           source_include=source_include)

In [6]:
len(train_data_error)

804

In [7]:
frames=[train_data_normal,train_data_error]
train_data=pd.concat(frames)

In [8]:
len(train_data)

2527

In [9]:
train_data.info()

In [10]:
train_data.groupby('isException').count()

,id,logBody
isException,,
0,1723,1723
1,804,804


In [11]:
feature_data_array = train_data[logmsg]
class_data_array = train_data[tag]
print len(feature_data_array)
print len(class_data_array)

In [12]:
import nltk
import string
from nltk.corpus import stopwords
#from nltk.stem.porter import *

#训练数据关键词提取列表
def tokenize(text):
    f = open(r"E:\zxdf\ml\linkdata-log\dataset\testrecord\keyword_logbody.txt", "a+")
    tokens = nltk.word_tokenize(text)
    tokens2=[]
    for w in tokens:
        if w[0]=='-':
            tokens2.append(w[1:])
        elif w[0]=="'":
            tokens2.append(w[1:])    
        else:
            tokens2.append(w)
    
    stopword_tokens = [i for i in tokens2 if i not in string.punctuation]
    stopword_tokens2 = [w for w in stopword_tokens if w not in stopwords.words('english')]
    
    stopwords_custom=["''", "``","||","'/","'/'",u"'0x0","'2","'=","'s",'**','***','***.***','**constraint.unique_wt**','**failed','**method','--',
                      '-1','-297991290629036654','-614','-6182496564283649260','-6787310117729693199','-8','-999999999',u'-e',
                      '..','...','.\\xxx\\xxx.txt','/*','/**','/c','/p','0','1','==','b','c','e','x','===',
                      'v'
                     ]
    #,'mapped'
    stopword_tokens3 = [w for w in stopword_tokens2 if w not in stopwords_custom]    
            
    print >> f, "%s" %(stopword_tokens3) 
    f.close()
    return stopword_tokens3

In [13]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer=CountVectorizer(tokenizer=tokenize, stop_words='english', decode_error='ignore')
csr_mat = vectorizer.fit_transform(feature_data_array)
# 获取词袋模型中的所有词
wordlist = vectorizer.get_feature_names()  
# tf矩阵 元素a[i][j]表示j词在i类文本中的tf
countlist = csr_mat.toarray()

In [ ]:
#countlist[:,0:5]

In [14]:
import numpy as np
#feature_datas.toarray()
#class_data_array
wordcount_iserror_map = pd.DataFrame(data=np.c_[countlist, class_data_array],
                     columns=np.append(wordlist, ['isException']))

In [34]:
wordcount_iserror_map2=wordcount_iserror_map.apply(pd.to_numeric, errors='ignore')
#['mapped'].astype(int)
#.dtypes
#.info()
#.ix[:,20:30]

In [58]:
#wordcount_iserror_map2.dtypes
#for x in 
print wordcount_iserror_map2.isnull().sum()
#    print x

In [65]:
def count_word_in_doc(wordcount_iserror_map2,keyword):
    errornum=0
    normalnum=0
    for (k1),group in wordcount_iserror_map2.ix[:,[keyword,'isException']].groupby('isException'):
        print '%s' % '=========len========================='        
        print '%s' % k1,':',len(group)
        #print (group)
        if k1==0:
            for i in group.index:
                if group[keyword][i]>0.0:
                           normalnum += 1
        elif k1==1:
            for j in group.index:
                if group[keyword][j]> 0.0: 
                    errornum += 1
        else:
            print 'this is a error'
    
    print '%s' % '=========count_word_in_doc========================='        
    print '%s' % '1:',errornum
    print '%s' % '0:',normalnum

In [67]:
count_word_in_doc(wordcount_iserror_map2,'null')

In [ ]:
# 提取特征
#from sklearn.feature_extraction.text import TfidfTransformer
#tf_transformer=TfidfTransformer()
#feature_datas= tf_transformer.fit_transform(csr_mat)
#
## 获取词袋模型中的所有词
##tfidfwordlist =tf_transformer.get_feature_names()
## tf-idf矩阵 元素a[i][j]表示j词在i类文本中的tf-idf权重
#weighttlist = feature_datas.toarray()
from app.lib import nltk_text_analyze_lib as textAnalyzer
from sklearn.feature_extraction.text import TfidfVectorizer
tf_transformer = TfidfVectorizer(tokenizer=textAnalyzer.tokenize, stop_words='english', decode_error='ignore')
feature_datas = tf_transformer.fit_transform(feature_data_array)

In [ ]:
#feature_datas.toarray()[:,0:5]

In [ ]:
import numpy as np
#feature_datas.toarray()
#class_data_array
word_iserror_map = pd.DataFrame(data=np.c_[feature_datas.toarray(), class_data_array],
                     columns=np.append(tf_transformer.get_feature_names(), ['isException']))

In [ ]:
word_iserror_map = pd.DataFrame(data=np.c_[feature_datas.toarray(), class_data_array],
                     columns=np.append(tf_transformer.get_feature_names(), ['isException']))

In [ ]:
word_iserror_map.info()

In [ ]:
#word_iserror_map.iloc[:].groupby('isException').sum()
word_iserror_map.ix[:,['mapped','null','isException']].groupby('isException').sum()
#.groupby('isException').count()

In [ ]:
# 检查数据是否有缺失
#word_iserror_map.isnull().sum()

In [ ]:
# 观察样本中按类别数量是否比较均衡
word_iserror_map.groupby('isException').count()

In [ ]:
#from sklearn.model_selection import train_test_split
#from sklearn.preprocessing import LabelEncoder
#
#le = LabelEncoder()
#train_labelValues = le.fit_transform(class_data_array)
#
##, random_state=4
#X_train, X_test, y_train, y_test = train_test_split(feature_datas,train_labelValues,test_size=0.25)
#
##from sklearn.metrics import accuracy_score
#from sklearn import metrics
#
#model = DecisionTreeClassifier()
#model.fit(X_train, y_train)
#
#y_pred_on_test = model.predict(X_test)
#
#print(metrics.precision_score(y_test, y_pred_on_test, average=None))
#print(metrics.recall_score(y_test, y_pred_on_test, average='macro'))
#print(metrics.f1_score(y_test, y_pred_on_test, average='weighted'))

In [ ]:
#验证算法
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
def compareAlgorithm(X_train,Y_train,num_folds=10, seed=7, scoring='accuracy'):
    models = {}
    models['KNN'] = KNeighborsClassifier()
    models['CART'] = DecisionTreeClassifier()
    #models['SVM'] = SVC()
    #models['LR'] = LogisticRegression()
    # models['LDA'] = LinearDiscriminantAnalysis()
    # models['NB'] = GaussianNB()
    # 评估算法
    results = []
    for key in models:
        kfold = KFold(n_splits=num_folds, random_state=seed)
        cv_results = cross_val_score(models[key], X_train, Y_train, cv=kfold, scoring=scoring)
        results.append(cv_results)
        print('%s: %f (%f)' %(key, cv_results.mean(), cv_results.std()))
    return models, results

In [ ]:
compareAlgorithm(X_train=feature_datas, Y_train=class_data_array)

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.externals import joblib

le = LabelEncoder()
train_labelValues = le.fit_transform(class_data_array)

model = DecisionTreeClassifier()
model.fit(feature_datas, train_labelValues)

# teststack6678_804
#app/rules/modelspkl
#D:\zxdf\Workspaces\PycharmProjects\flask_ml\app\rules\modelspkl
joblib.dump(tf_transformer, r'D:\zxdf\Workspaces\PycharmProjects\flask_ml\app\rules\modelspkl\teststack6678_804\knowledge_tf_transformer.pkl')
joblib.dump(le, r'D:\zxdf\Workspaces\PycharmProjects\flask_ml\app\rules\modelspkl\teststack6678_804\knowledge_labelencoder.pkl')
joblib.dump(model, r'D:\zxdf\Workspaces\PycharmProjects\flask_ml\app\rules\modelspkl\teststack6678_804\knowledge_cart.pkl')

In [ ]:
#model = DecisionTreeClassifier()
#model.fit(feature_datas, class_data_array)

In [ ]:
testline='-Mapped java.lang.StringIndexOutOfBoundsException: String index out of range: 0 '

test_datas = []
featureValue = testline.lower().strip()
test_datas.append(featureValue)
#test_feature_datas = tf_transformer.transform(test_datas)
test_vect_datas=vectorizer.transform(test_datas)
test_feature_datas= tf_transformer.transform(test_vect_datas)
model.predict(test_feature_datas)